In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/resnet-weights/pytorch/default/1/resnet18-f37072fd.pth
/kaggle/input/dog-vs-cat-vs-bird/dataset/sample_submission.csv
/kaggle/input/dog-vs-cat-vs-bird/dataset/test/img_1963.png
/kaggle/input/dog-vs-cat-vs-bird/dataset/train/cat_9146.png


## Baseline Model

In [3]:
import pandas as pd
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from PIL import Image

In [4]:
import torch.nn as nn
import torch.optim as optim

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
train_csv = pd.read_csv('/kaggle/input/dog-vs-cat-vs-bird/dataset/train.csv')
print(train_csv['label'].unique())  # Ensure this outputs only [0, 1, 2]

['cat' 'dog' 'bird']


In [49]:
images, labels = next(iter(train_loader))
outputs = model(images.to(device))
print(f"Model output shape: {outputs.shape}")  # Should be [batch_size, 3]
print(f"Labels: {labels}")

Model output shape: torch.Size([64, 3])
Labels: ('cat', 'cat', 'cat', 'bird', 'bird', 'cat', 'cat', 'bird', 'bird', 'cat', 'cat', 'bird', 'cat', 'dog', 'bird', 'cat', 'cat', 'dog', 'bird', 'cat', 'bird', 'cat', 'bird', 'cat', 'cat', 'cat', 'bird', 'cat', 'dog', 'bird', 'cat', 'dog', 'bird', 'bird', 'bird', 'bird', 'dog', 'cat', 'cat', 'cat', 'cat', 'cat', 'dog', 'bird', 'bird', 'bird', 'bird', 'dog', 'dog', 'cat', 'bird', 'dog', 'bird', 'cat', 'bird', 'bird', 'dog', 'bird', 'cat', 'cat', 'dog', 'cat', 'dog', 'bird')


In [7]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        # Read CSV and store the image data and labels
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
        # Initialize the LabelEncoder to convert categorical labels to numeric
        self.label_encoder = LabelEncoder()
        # Fit the encoder on all unique labels
        self.data.iloc[:, 1] = self.label_encoder.fit_transform(self.data.iloc[:, 1])

    def __len__(self):
        # Return the total number of samples
        return len(self.data)

    def __getitem__(self, idx):
        # Get the image path and label
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])  # Image filename
        image = Image.open(img_name).convert('RGB')  # Open and convert to RGB
        label = self.data.iloc[idx, 1]  # Categorical label (already encoded to integer)

        # Apply transformation if any (e.g., resize, normalize)
        if self.transform:
            image = self.transform(image)

        return image, label

    def get_label_encoder(self):
        # Return the fitted label encoder
        return self.label_encoder

In [78]:
img_path = '/kaggle/input/dog-vs-cat-vs-bird/dataset/train/cat_9146.png'
image = Image.open(img_path)
print(f"Original Image Mode: {image.mode}")  # Should show 'RGBA'

image = image.convert('RGB')
print(f"Converted Image Mode: {image.mode}")  # Should show 'RGB'

Original Image Mode: RGBA
Converted Image Mode: RGB


In [8]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [9]:
train_dataset = CustomDataset(
    csv_file='/kaggle/input/dog-vs-cat-vs-bird/dataset/train.csv',
    root_dir='/kaggle/input/dog-vs-cat-vs-bird/dataset/train',
    transform=transform
)

In [10]:
label_encoder = train_dataset.get_label_encoder()

In [11]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [82]:
images, labels = next(iter(train_loader))
print(f"Image batch shape: {images.shape}")  # Should be [batch_size, 3, height, width]

Image batch shape: torch.Size([64, 3, 32, 32])


In [60]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 8 * 8, 128),
            nn.ReLU(),
            nn.Linear(128, 3)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

NameError: name 'SimpleCNN' is not defined

In [64]:
def train_model(model, train_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_loss, correct = 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()

        print(f"Epoch {epoch+1}/{epochs} | Loss: {train_loss/len(train_loader):.4f}, Accuracy: {correct/len(train_dataset):.4f}")

In [65]:
train_model(model, train_loader, epochs=10)

Epoch 1/10 | Loss: 0.9269, Accuracy: 0.5503
Epoch 2/10 | Loss: 0.7895, Accuracy: 0.6435
Epoch 3/10 | Loss: 0.7213, Accuracy: 0.6809
Epoch 4/10 | Loss: 0.6542, Accuracy: 0.7212
Epoch 5/10 | Loss: 0.5996, Accuracy: 0.7479
Epoch 6/10 | Loss: 0.5413, Accuracy: 0.7727
Epoch 7/10 | Loss: 0.4826, Accuracy: 0.7993
Epoch 8/10 | Loss: 0.4106, Accuracy: 0.8364
Epoch 9/10 | Loss: 0.3485, Accuracy: 0.8643
Epoch 10/10 | Loss: 0.2749, Accuracy: 0.8960


In [16]:
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_names = sorted(os.listdir(root_dir))  # List of image filenames in the test directory
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_names[idx])  # Path to the image
        image = Image.open(img_name).convert('RGB')  # Convert to RGB to handle RGBA images

        if self.transform:
            image = self.transform(image)  # Apply transformations if any

        return image, self.image_names[idx]  # Return image and filename for tracking

In [17]:
test_dataset = TestDataset(root_dir='/kaggle/input/dog-vs-cat-vs-bird/dataset/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [71]:
model.eval()
predictions = []
filenames = []

with torch.no_grad():
    for images, file_names in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = outputs.argmax(1).cpu().numpy()  # Get class predictions (indices)
        predictions.extend(preds)
        filenames.extend(file_names)  # Keep track of filenames

In [85]:
predicted_labels = label_encoder.inverse_transform(predictions)

In [112]:
submission_df = pd.DataFrame({
    'filename': filenames,
    'label': predicted_labels
})

In [113]:
submission_df.head()

,filename,label
0,img_0.png,dog
1,img_1.png,dog
2,img_10.png,cat
3,img_100.png,cat
4,img_1000.png,cat


In [114]:
submission_df.to_csv('/kaggle/working/submission.csv', index=False)

In [115]:
submission_df = pd.read_csv('/kaggle/working/submission.csv')
print(submission_df.head())

       filename label
0     img_0.png   dog
1     img_1.png   dog
2    img_10.png   cat
3   img_100.png   cat
4  img_1000.png   cat


In [116]:
sample_submission_df = pd.read_csv('/kaggle/input/dog-vs-cat-vs-bird/dataset/sample_submission.csv')
print(sample_submission_df.head())

    filename label
0  img_0.png   cat
1  img_1.png   dog
2  img_2.png  bird
3  img_3.png   cat
4  img_4.png   dog


## Optimization

### Dropout

In [ ]:
class SimpleCNNWithDropout(nn.Module):
    def __init__(self):
        super(SimpleCNNWithDropout, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),  # Dropout with 25% rate
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 8 * 8, 128),
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout with 50% rate
            nn.Linear(128, 3)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

In [ ]:

model = SimpleCNNWithDropout().to(device)

## Learning Rate Scheduler

In [ ]:
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)  # Reduce LR by 10% every 5 epochs

def train_model_with_scheduler(model, train_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_loss, correct = 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()

        scheduler.step()  # Adjust learning rate
        print(f"Epoch {epoch+1}/{epochs} | Loss: {train_loss/len(train_loader):.4f}, Accuracy: {correct/len(train_dataset):.4f}")

In [ ]:
train_model_with_scheduler(model, train_loader, epochs=10)

## Data Augmentation

In [ ]:
transform_with_augmentation = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [ ]:
train_dataset = CustomDataset(
    csv_file='/kaggle/input/dog-vs-cat-vs-bird/dataset/train.csv',
    root_dir='/kaggle/input/dog-vs-cat-vs-bird/dataset/train',
    transform=transform_with_augmentation
)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd
import os

# New model definition with Dropout
class SimpleCNNWithDropout(nn.Module):
    def __init__(self):
        super(SimpleCNNWithDropout, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),  # Dropout with 25% rate
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 8 * 8, 128),
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout with 50% rate
            nn.Linear(128, 3)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

# Data transformation with augmentation
transform_with_augmentation = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Update train dataset with augmented transformations
train_dataset = CustomDataset(
    csv_file='/kaggle/input/dog-vs-cat-vs-bird/dataset/train.csv',
    root_dir='/kaggle/input/dog-vs-cat-vs-bird/dataset/train',
    transform=transform_with_augmentation
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model, optimizer, and criterion
model = SimpleCNNWithDropout().to(device)  # Use the new model with dropout
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [12]:
# Set up the learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)  # Reduce LR by 10% every 5 epochs

# Training function with scheduler
def train_model_with_scheduler(model, train_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_loss, correct = 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()

        scheduler.step()  # Adjust learning rate
        print(f"Epoch {epoch+1}/{epochs} | Loss: {train_loss/len(train_loader):.4f}, Accuracy: {correct/len(train_dataset):.4f}")

In [13]:
# Train the model
train_model_with_scheduler(model, train_loader, epochs=10)

Epoch 1/10 | Loss: 1.0506, Accuracy: 0.4457
Epoch 2/10 | Loss: 1.0130, Accuracy: 0.4773
Epoch 3/10 | Loss: 0.9934, Accuracy: 0.5028
Epoch 4/10 | Loss: 0.9848, Accuracy: 0.5104
Epoch 5/10 | Loss: 0.9853, Accuracy: 0.5097
Epoch 6/10 | Loss: 0.9616, Accuracy: 0.5282
Epoch 7/10 | Loss: 0.9608, Accuracy: 0.5269
Epoch 8/10 | Loss: 0.9482, Accuracy: 0.5401
Epoch 9/10 | Loss: 0.9526, Accuracy: 0.5344
Epoch 10/10 | Loss: 0.9456, Accuracy: 0.5407


In [18]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Same normalization as training
])

# Create the test dataset and loader
test_dataset = TestDataset(
    root_dir='/kaggle/input/dog-vs-cat-vs-bird/dataset/test',
    transform=transform
)

test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [19]:
model.eval()  # Set the model to evaluation mode

SimpleCNNWithDropout(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.25, inplace=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.25, inplace=False)
  )
  (fc): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4096, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=128, out_features=3, bias=True)
  )
)

In [23]:
predictions = []
filenames = []

# Make predictions
with torch.no_grad():  # No need to compute gradients for inference
    for images, file_names in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = outputs.argmax(1).cpu().numpy()  # Get the predicted class labels (as integers)
        predictions.extend(preds)
        filenames.extend(file_names)  # Keep track of the filenames for the results

# Convert the predictions to actual labels using the label encoder (if necessary)
predicted_labels = label_encoder.inverse_transform(predictions)

# Create a DataFrame to store the results
submission_df = pd.DataFrame({
    'filename': filenames,
    'label': predicted_labels
})

# Optionally, save the predictions to a CSV file for submission
submission_df.to_csv('/kaggle/working/submission_optimized.csv', index=False)

# Check the first few rows of the submission DataFrame
print(submission_df.head())

       filename label
0     img_0.png   cat
1     img_1.png   dog
2    img_10.png   cat
3   img_100.png   cat
4  img_1000.png  bird


## ResNet

In [12]:
# Step 1: Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import os
import pandas as pd

# Step 2: Define the Custom Dataset for training
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

        # Initialize the LabelEncoder if necessary
        self.label_encoder = LabelEncoder()
        self.data.iloc[:, 1] = self.label_encoder.fit_transform(self.data.iloc[:, 1])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        label = self.data.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        return image, label

# Step 3: Define transformations for training and testing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match the input size for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Step 4: Load your train dataset
train_dataset = CustomDataset(
    csv_file='/kaggle/input/dog-vs-cat-vs-bird/dataset/train.csv',
    root_dir='/kaggle/input/dog-vs-cat-vs-bird/dataset/train',
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [13]:
# Step 5: Load pre-trained ResNet-18 model
resnet = models.resnet18(pretrained=False)  # Initialize without downloading
weights_path = "/kaggle/input/resnet-weights/pytorch/default/1/resnet18-f37072fd.pth"  # Provide path to the manually downloaded weights file
resnet.load_state_dict(torch.load(weights_path))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-13-c1a7139053cd>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions

<All keys matched successfully>

In [14]:
# Modify the final fully connected layer to match your number of classes (3)
resnet.fc = nn.Linear(resnet.fc.in_features, 3)

# Freeze all layers except the final fully connected layer
for param in resnet.parameters():
    param.requires_grad = False

# Only the parameters of the final fully connected layer will require gradients
for param in resnet.fc.parameters():
    param.requires_grad = True

# Step 6: Move the model to the appropriate device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet = resnet.to(device)

# Step 7: Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.fc.parameters(), lr=0.001)

In [15]:
# Step 8: Train the model
def train_model(model, train_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss, correct = 0.0, 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        # Print training results
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {correct/total:.4f}")

In [16]:
# Train the model
train_model(resnet, train_loader, epochs=10)

# Step 9: Save the trained model
torch.save(resnet.state_dict(), 'resnet_finetuned.pth')

Epoch [1/10], Loss: 0.6402, Accuracy: 0.7321


KeyboardInterrupt: 

In [17]:
# Step 10: Testing the model (for images without CSV file)
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_names = sorted(os.listdir(root_dir))  # List of image filenames in the test directory
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_names[idx])
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, self.image_names[idx]

# Load the test dataset (images only, no labels)
test_dataset = TestDataset(root_dir='/kaggle/input/dog-vs-cat-vs-bird/dataset/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Set model to evaluation mode for testing
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
predictions = []
filenames = []

# Make predictions
with torch.no_grad():
    for images, file_names in test_loader:
        images = images.to(device)
        outputs = resnet(images)
        preds = outputs.argmax(1).cpu().numpy()  # Get predicted class labels
        predictions.extend(preds)
        filenames.extend(file_names)

# Convert predictions to class labels
label_encoder = train_dataset.label_encoder
predicted_labels = label_encoder.inverse_transform(predictions)

# Save the results to a CSV file
submission_df = pd.DataFrame({
    'filename': filenames,
    'label': predicted_labels
})

In [20]:
submission_df.to_csv('/kaggle/working/submission_resnet.csv', index=False)

## Fine Tuning Transfer Learning

In [21]:
# Unfreeze selected layers
for name, param in resnet.named_parameters():
    if "layer4" in name:  # Unfreeze the last residual block
        param.requires_grad = True

In [22]:
optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.0001)

In [23]:
resnet.eval()
predictions, filenames = [], []

with torch.no_grad():
    for images, file_names in test_loader:
        images = images.to(device)
        outputs = resnet(images)
        preds = outputs.argmax(1).cpu().numpy()
        predictions.extend(preds)
        filenames.extend(file_names)

In [24]:
predicted_labels = label_encoder.inverse_transform(predictions)
submission_df = pd.DataFrame({'filename': filenames, 'label': predicted_labels})
submission_df.to_csv('/kaggle/working/submission_resnet_optimization.csv', index=False)